In [126]:
ASSETS_DIR = "/content/drive/MyDrive/mockotify_assets"

In [127]:
import os

import pandas as pd
import numpy as np

In [128]:
metadata = pd.read_csv(ASSETS_DIR + "/dataset.csv")
df = metadata.copy()

drop_tables = ["Unnamed: 0", "explicit", "key", "mode"]
df = df.drop(drop_tables, axis=1)

df['popularity'] = df["popularity"].apply(lambda _: int(_/5))

In [129]:
#drop_tables = ["Unnamed: 0", "explicit", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature"]

In [130]:
df["track_genre"] = df.groupby("track_id")["track_genre"].transform(lambda x: ";".join(x))


In [131]:
df = df.drop_duplicates(subset=["track_id"])

In [132]:
df

,track_id,artists,album_name,track_name,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,14,230666,0.676,0.4610,-6.746,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic;j-pop;singer-songwriter;songwriter
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,11,149610,0.420,0.1660,-17.235,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic;chill
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,11,210826,0.438,0.3590,-9.734,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,14,201933,0.266,0.0596,-18.515,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,16,198853,0.618,0.4430,-9.681,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,4,384999,0.172,0.2350,-16.393,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,4,385000,0.174,0.1170,-18.318,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,4,271466,0.629,0.3290,-10.895,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,8,283893,0.587,0.5060,-10.889,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [165]:
def get_dummies(df, column):
  t = pd.get_dummies(df[column])
  columns = t.columns
  t.columns = [f"{column};{_}" for _ in columns]
  t.reset_index(drop=True, inplace=True)

  return t

In [166]:
from sklearn.feature_extraction.text import TfidfVectorizer

genre_tfidf = TfidfVectorizer()
genre_X = genre_tfidf.fit_transform(df["track_genre"])
genre_df = pd.DataFrame(genre_X.toarray())
genre_df.columns = ["gen;" + _ for _ in genre_tfidf.get_feature_names()]

#d = ["popularity", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]

#for i in d:
#  i_d = get_dummies(df, i)
#  i_d.to_parquet(ASSETS_DIR + "/output_d/" + i + "_d")

pop_d = get_dummies(df, "popularity")
res = pd.concat([genre_df, pop_d], axis=1)
res["track_id"] = df["track_id"].values
res

res.to_parquet(ASSETS_DIR + "tfidf")

#sim_df = cosine_similarity(res, dense_output=True)
#sim_df.head()



/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [167]:
from sklearn.metrics.pairwise import cosine_similarity

tfidf_5000 = pd.read_parquet(ASSETS_DIR + "tfidf").sample(5000)
ids = tfidf_5000[:5000]["track_id"]
M = pd.DataFrame(cosine_similarity(tfidf_5000.iloc[:, :-1]), columns=ids)
M = M.set_index(ids)
M

track_id,7LS9Fhrr9TdKZW9ow444uS,7IVzMMlDsh4W4mIavGdKi0,026qbtwLI1UtOFQ22zqY2t,45ai35oxYWwa4LAjEUvQNC,6EqRZzXK89sFBZBxLRPTZe,5my3f3dMZENvUwAXYVQNmO,0IT2exfz39cCkA1Jl8GRiv,4gCmdnNpWVzfgcprEL2Mew,4Ds4bq6aqOSAp1T7DikOi4,0XmvrI2Fczr9zkvgQvqeNx,...,4Cf4x7lxls5ksUs8YLCoKL,5hXGYD3ItfFllTsp8t0Up6,7yNvhnnFM1Q5RbPyT5HNDE,2wtQMLH8JWg6TrAcxrmcy3,44l7z2oUGyC7YovlvLQ133,3dlWabGcrqGU349TrJ9V6g,3vJJUrkIfbWhH2DwoquzjX,5DPJ4y8ArSycYrlm08L1Eq,4SKRNfT7WI73MJho3suqYl,53sOR02nAN4tpAzAkL2EnB
track_id,,,,,,,,,,,,,,,,,,,,,
7LS9Fhrr9TdKZW9ow444uS,1.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.5,0.0,0.5
7IVzMMlDsh4W4mIavGdKi0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.5,0.0,0.0,0.0,0.0,0.0
026qbtwLI1UtOFQ22zqY2t,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.5,0.5,0.5,0.000000,0.0,0.0,0.5,0.0,0.0,0.0
45ai35oxYWwa4LAjEUvQNC,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
6EqRZzXK89sFBZBxLRPTZe,0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.5,0.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3dlWabGcrqGU349TrJ9V6g,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,...,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0
3vJJUrkIfbWhH2DwoquzjX,0.0,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,...,0.5,0.5,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0
5DPJ4y8ArSycYrlm08L1Eq,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.5


In [ ]:
M.to_parquet()

In [ ]:
os.mkdir(ASSETS_DIR + "/OUTPUT/")

In [168]:
M.to_parquet(ASSETS_DIR + "/OUTPUT/m")

In [177]:
pd.read_parquet(ASSETS_DIR + "/OUTPUT/m")

track_id,7LS9Fhrr9TdKZW9ow444uS,7IVzMMlDsh4W4mIavGdKi0,026qbtwLI1UtOFQ22zqY2t,45ai35oxYWwa4LAjEUvQNC,6EqRZzXK89sFBZBxLRPTZe,5my3f3dMZENvUwAXYVQNmO,0IT2exfz39cCkA1Jl8GRiv,4gCmdnNpWVzfgcprEL2Mew,4Ds4bq6aqOSAp1T7DikOi4,0XmvrI2Fczr9zkvgQvqeNx,...,4Cf4x7lxls5ksUs8YLCoKL,5hXGYD3ItfFllTsp8t0Up6,7yNvhnnFM1Q5RbPyT5HNDE,2wtQMLH8JWg6TrAcxrmcy3,44l7z2oUGyC7YovlvLQ133,3dlWabGcrqGU349TrJ9V6g,3vJJUrkIfbWhH2DwoquzjX,5DPJ4y8ArSycYrlm08L1Eq,4SKRNfT7WI73MJho3suqYl,53sOR02nAN4tpAzAkL2EnB
track_id,,,,,,,,,,,,,,,,,,,,,
7LS9Fhrr9TdKZW9ow444uS,1.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.5,0.0,0.5
7IVzMMlDsh4W4mIavGdKi0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.5,0.0,0.0,0.0,0.0,0.0
026qbtwLI1UtOFQ22zqY2t,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.5,0.5,0.5,0.000000,0.0,0.0,0.5,0.0,0.0,0.0
45ai35oxYWwa4LAjEUvQNC,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
6EqRZzXK89sFBZBxLRPTZe,0.5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.5,0.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3dlWabGcrqGU349TrJ9V6g,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,...,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0
3vJJUrkIfbWhH2DwoquzjX,0.0,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,...,0.5,0.5,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0
5DPJ4y8ArSycYrlm08L1Eq,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.5


In [170]:
import base64
import requests
import datetime

client_id = "4f1b4af6d89c47a89a38fd7ca87bc0bc"
client_secret = "f18c0e3f3a924c42be997ad9614746ee"

client_cred = f"{client_id}:{client_secret}"
client_cred_b64 = base64.b64encode(client_cred.encode())


token_url = "https://accounts.spotify.com/api/token"
token_data = {
    "grant_type": "client_credentials"
}
token_headers = {
    "Authorization": f"Basic {client_cred_b64.decode()}",
    ""
}

def refresh_token():
  res = requests.post(token_url, data=token_data, headers=token_headers)
  token = res.json()["access_token"]

  return token

In [184]:
l = list(filter(lambda x: x.get("id"), a))
ids = list(map(lambda x: x["id"], l))

m = m.loc[ids]
m = m[ids]
m.to_parquet(ASSETS_DIR + "/final_matrix")

In [185]:
len(l)

2993

In [ ]:
import time
from tqdm import tqdm

data = []
token = refresh_token()

for i in tqdm(ids):
  def inner():
    resp = requests.get(f"https://api.spotify.com/v1/tracks/{i}", headers={
      "Authorization": f"Bearer {token}"
    })

    if resp.status_code == 401:
      refresh_token()
      inner()

    if resp.status_code == 429:
      t = int(resp.headers["Retry-After"])
      print("SLEEPING...", t)
      time.sleep(t)
      inner()

    r = resp.json()

    if r.get("available_markets"):
      del r["available_markets"]
  
    if r.get("album"):
      if r.get("album").get("available_markets"):
        del r["album"]["available_markets"]

    data.append(r)
    time.sleep(2)
  inner()

In [122]:
import json

with open(ASSETS_DIR + '/final_data.json', 'w', encoding='utf-8') as f:
  json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
m = pd.read_parquet(ASSETS_DIR + "/OUTPUT/m")
l = list(m.index)

In [194]:
genres = df[df["track_id"].isin(list(m.index))][["track_id", "track_genre"]]

for i in l:
  i["genre"] = str(genres[genres["track_id"] == i["id"]]["track_genre"].values[0])

In [197]:
with open(ASSETS_DIR + '/final_data.json', 'r', encoding='utf-8') as f:
  a = json.load(f)

In [214]:
genres["genre_list"] = genres.apply(lambda x: x["track_genre"].split(";"), axis=1)

In [227]:
genres.explode("genre_list").drop_duplicates("genre_list")["genre_list"].reset_index(drop=True).to_json(ASSETS_DIR + "/genres.json", orient="index")

In [228]:
m

track_id,30QU4RJntz98HlLOlkRgew,2I4hVGTtVC6zl8d4vbrmbd,0A496x5Uwuivwvive8jEsl,0uTfxpuGPeb5xvkjduxQT3,1FRVpTcTnGKu8x1xEJl4xf,5TFfKIjipFrTMBKca4UMvF,3Pzrqke0TSziYx20MT26KG,3m2jKnyeVKdTDFphqa2vou,669cOBjpY4Y5O98Nmk6139,4Dp3yrEK6dQzr9oM2UtZgR,...,0gEaeqVRHPzRc7HMXtOKc7,1onuiNSXonc5vOJU9igdFk,6FzjJPwf5JzzFOei7h6Qrp,5H9bGkHgHgtsKxZhg15MCU,54tHbuVRyr4HwyEvpBdi4k,3LRMSQH09zCPUXI9vJCZZx,573mudwDhftRSSbvLHC8xS,1kh6iEcYhNL6jbMr5BVEVG,0JiPUFfWlhT1SJ92eZ84GU,47VMbtOpN8cHnDFboLJWLz
track_id,,,,,,,,,,,,,,,,,,,,,
30QU4RJntz98HlLOlkRgew,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2I4hVGTtVC6zl8d4vbrmbd,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
0A496x5Uwuivwvive8jEsl,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.5,0.0,0.0,0.0,0.0
0uTfxpuGPeb5xvkjduxQT3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.117964,0.0,0.0,0.151521,0.0,0.0,0.0,0.0,0.0,0.0
1FRVpTcTnGKu8x1xEJl4xf,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.5,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3LRMSQH09zCPUXI9vJCZZx,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0
573mudwDhftRSSbvLHC8xS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.000000,...,0.500000,0.5,0.5,0.000000,0.0,0.0,1.0,0.0,0.0,0.0
1kh6iEcYhNL6jbMr5BVEVG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0
